In [24]:
import pandas as pd
import numpy as np
import pandas as pd
from docxtpl import DocxTemplate,  InlineImage
from docx.shared import Mm
import sys
import mysql.connector
import os
from dotenv import load_dotenv
from datetime import date
# id=sys.argv[1]
id=82
today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host=DB_HOST,
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
cotizacion=pd.read_sql("select * from quotations where id ="+str(id),cnx)
cliente=pd.read_sql("""select * from customers where customers.id="""+str(cotizacion['customer_id'].values[0]),cnx)
user=pd.read_sql("""select * from users where users.id="""+str(cotizacion['user_id'].values[0]),cnx)
productos=pd.read_sql("""select * from cart_products where cart_products.quotation_id="""+str(cotizacion['id'].values[0]),cnx)
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
year = date.strftime("%Y")

from tablas_dict import tablas
aceros=pd.read_sql('select * from steels ',cnx)
aceros.loc[aceros['caliber']=='EST 3 IN','caliber']='EST3'

products=pd.DataFrame()
#iterar sobre tablas
for i in tablas:
    print(i)
    #buscar en la base de datos todos los productos de esta tabla
    #pertenecientes a la cotizacion pedida por el usuario.
    p=pd.read_sql('select * from '+i+' where quotation_id = '+str(id),cnx)
    p=p.assign(tabla=i)
    if(('cost' not in p.columns)&(len(p)>0)):
        if('caliber' not in p.columns):
             #esto es en especifico por un caso en que todas kas piezas son cal 14
             p=p.assign(caliber='14')
        try:
            p['caliber']=p['caliber'].str.replace('-','')
        except:
            print(' ')
        print(str(p['caliber'].values[0]))
        costo=aceros.loc[aceros['caliber']==str(p['caliber'].values[0]),'cost'].values[0]
        if('total_kg' in p.columns):
            p=p.assign(cost=costo*p.total_kg)
        if('total_weight' in p.columns):
           
            p=p.assign(cost=costo*p.total_weight)
        if('weight_kg' in p.columns):
         
            p=p.assign(cost=costo*p.weight_kg)
        if('weight' in p.columns):
          
            p=p.assign(cost=costo*p.weight)
        if('long' in p.columns):
           
            p=p.assign(cost=costo*p.long)
        print(i)
    products=products.append(p,ignore_index=True)
cols_to_fill_str=['description','protector','model','sku']
products[cols_to_fill_str]=products[cols_to_fill_str].fillna('')
cols_kg=['weight','total_kg','total_weight','weight_kg']
cols_m2=['m2','total_m2']
products[cols_kg+cols_m2]=products[cols_kg+cols_m2].fillna(0)
price_cols=['price','total_price','import','unit_price']
products[price_cols] =products[price_cols].fillna(0)    
pricelist_protectors=pd.read_sql('select * from price_list_protectors',cnx)
quotation_protectors=pd.read_sql('select quotation_protectors.*, protectors.sku from quotation_protectors  inner join protectors on protectors.protector=quotation_protectors.protector where quotation_id ='+str(id),cnx)
quotation_shlf=pd.read_sql('select * from selective_heavy_load_frames where quotation_id ='+str(id),cnx)
materials=pd.read_sql('select * from (materials left join price_list_screws on materials.price_list_screw_id= price_list_screws.id)left join price_lists on price_lists.id=materials.price_list_id',cnx)
materials['type']=materials['type'].fillna('')

doc = DocxTemplate("plantilla.docx")
productos=[]
for i in range(len(products)):
    productos.append({'nombre':tablas[products['tabla'].values[3]],'precio':products[price_cols].sum(axis=1).values[i],'cantidad':products['amount'].values[i]})


c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site

double_deep_crossbars
double_deep_floors
double_deep_floor_reinforcements
double_deep_heavy_load_frames
double_deep_joist_box25s
double_deep_joist_box25_caliber14s
double_deep_joist_box2s
double_deep_joist_box2_caliber14s
double_deep_joist_c2_s
double_deep_joist_chairs
double_deep_joist_l25_caliber14s
double_deep_joist_l25_s
double_deep_joist_l2_caliber14s
double_deep_joist_l2_s
double_deep_joist_lrs
double_deep_joist_structurals
double_deep_miniature_frames
double_deep_spacers
double_deep_structural_frames
freights
grills
installations
packagings
quot25_j_galvanized_panels
 
14
quot25_j_galvanized_panels
quot25_j_painted_panels
 
14
quot25_j_painted_panels
quot2_j_galvanized_panels
 
14
quot2_j_galvanized_panels
quot2_j_painted_panels
 
14
quot2_j_painted_panels
quotation_administratives
quotation_installs
quotation_protectors
quotation_specials


C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_6208\1140653343.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_6208\1140653343.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase 

quotation_travel_assignments
quotation_uninstalls
quot_chair_j_galvanized_panels
 
14
quot_chair_j_galvanized_panels
quot_chair_j_painted_panels
 
14
quot_chair_j_painted_panels
selective_crossbars
14
selective_crossbars
selective_floors
14
selective_floors
selective_floor_reinforcements
14
selective_floor_reinforcements
selective_heavy_load_frames
14
selective_heavy_load_frames
selective_joist_box25s
10
selective_joist_box25s
selective_joist_box25_caliber14s
14
selective_joist_box25_caliber14s
selective_joist_box2s
10
selective_joist_box2s
selective_joist_box2_caliber14s
14
selective_joist_box2_caliber14s
selective_joist_c2_s
10
selective_joist_c2_s
selective_joist_chairs
10
selective_joist_chairs
selective_joist_l25_caliber14s
14
selective_joist_l25_caliber14s
selective_joist_l25_s
10
selective_joist_l25_s
selective_joist_l2_caliber14s
14
selective_joist_l2_caliber14s
selective_joist_l2_s
10
selective_joist_l2_s
selective_joist_lrs
16
selective_joist_lrs
selective_joist_structurals
E

C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_6208\1140653343.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\fgonzalez\AppData\Local\Temp\ipykernel_6208\1140653343.py:77: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products=products.append(p,ignore_index=True)
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase 

In [40]:
productos=[]
for i in range(len(products)):
    productos.append({'nombre':tablas[products['tabla'].values[3]],'precio':products[price_cols].sum(axis=1).values[i],'cantidad':products['amount'].values[i]})


In [38]:
productos=[]
productos.append({'jeje':34})

In [41]:
productos

[{'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 0.0,
  'cantidad': 1},
 {'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 3.42,
  'cantidad': 1},
 {'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 6.84,
  'cantidad': 2},
 {'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 910.23,
  'cantidad': 1},
 {'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 1820.45,
  'cantidad': 2},
 {'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 725.71,
  'cantidad': 1},
 {'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 938.5,
  'cantidad': 1},
 {'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 0.0,
  'cantidad': 1},
 {'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 51.29,
  'cantidad': 1},
 {'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 102.57,
  'cantidad': 2},
 {'nombre': 'PANELES GALVANIZADOS VIGA L DE 2.5 IN ',
  'precio': 1979.38,
  'cantidad': 1}

In [43]:
phi= (1 + 5 ** 0.5) / 2

In [45]:
1200-1200/phi

458.3592135001262